# Riskless lifetime spending

Here we figure out how to optimally spend a pot of money over a known time horizon, given access to a risk-free investment giving return $r_{ra}$ and a personal rate of time preference $r_{tp}$.

An optimal investment policy has these properties:

1. Spending more is better than spending less
1. There are decreasing marginal benefits to spending more
1. Spending should be as smooth as possible over time
1. Spending should react to changes in the value and quality of our investments, after tax and inflation-adjusted
1. Spending and investing should depend on our expected, but uncertain, personal longevity.

In [1]:
import numpy as np 
import polars as pl 
from polars import col 
from tqdm.autonotebook import tqdm
from findec import crra_utility

from IPython.display import display

/tmp/ipykernel_32765/1453797730.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Imagine you are 65 years old, and you've just retired with $1M. Assume you know you're going to live for exactly 20 years. How should we spend our money such that we maximize our expected lifetime utility, over our remaining 20 years?

The solution is to spend a fraction $c_t$ of wealth $W_t$ which maximizes the sum of all future year's discounted utility.

I.e. choose a $c_t$ at every year $t$ such that we maximize

$$\sum_{t=1}^T \frac{U(c_t W_t)}{(1+r_{tp})^t}$$

where $W_t = W_{t-1}(1-c_{t-1})(1+r_{ra})$

To simulate this, we use dynamic programming.

Admission: I got chatGPT o1 to solve this for me. Terrifyingly, it got basically the same results as Table 9.1 of Haghani & White.

In [2]:
initial_wealth = 1e6
GAMMA = 2.0
risk_adjusted_returns_per_year = 3e-2
future_utility_discount_rate_per_year = 1e-2
time_horizon = 20

In [3]:
def solve_consumption(
    *,
    W0: float,
    r_ra: float,
    r_tp: float,
    gamma: float,
    T: int,
    n_grid: int,
    c_grid_size: int,
):
    # 1. Build wealth grid
    W_max = W0 * (1 + r_ra) ** T * 2.0  # factor of 2 is just for some margin
    W_grid = np.linspace(1e-3, W_max, n_grid)

    # 2. Arrays for value and policy
    
    # V[t,i] holds the maximum discounted utility achievable from time t onward if your current wealth is W_grid[i]
    V = np.zeros((T + 2, n_grid)) 

    # C[t,i] holds the optimal consumption fraction that achieves that maximum
    C = np.zeros((T + 2, n_grid))

    # 3. Terminal condition: V[T+1] = 0
    # Already zero by default. No bequest.

    # 4. Precompute discount factors
    discount_factors = [(1 + r_tp) ** t for t in range(T + 2)]

    # 5. Discretize c in [0, 1] but skip exact zero
    c_candidates = np.linspace(0, 1, c_grid_size)

    # 6. Backwards induction: starting at the last decision point, working backwards
    for t in tqdm(reversed(range(1, T + 1)), total=T):
        df = discount_factors[t]
        for i, W_now in enumerate(W_grid):
            best_value = -np.inf
            best_c = 0

            # Evaluate every candidate consumption value
            for c_frac in c_candidates:
                # consumption in dollars
                cons = c_frac * W_now
                # next period wealth
                W_next = (W_now - cons) * (1 + r_ra)

                # clamp W_next to grid range
                if W_next < W_grid[0]:
                    W_next_index = 0
                elif W_next > W_grid[-1]:
                    W_next_index = n_grid - 1
                else:
                    # approximate nearest index
                    W_next_index = np.searchsorted(W_grid, W_next)
                    if W_next_index >= n_grid:
                        W_next_index = n_grid - 1

                # immediate utility
                immediate_utility = crra_utility(cons, gamma=gamma)
                # discount that immediate utility
                discounted_utility = immediate_utility / df

                # future value
                future_value = V[t + 1, W_next_index]

                total_value = discounted_utility + future_value

                if total_value > best_value:
                    best_value = total_value
                    best_c = c_frac

            # store the best
            V[t, i] = best_value
            C[t, i] = best_c
            
    # 7. Grab the optimal consumption fraction at t=1 for W0
    i_closest = np.argmin(np.abs(W_grid - W0))
    optimal_c_init = C[1, i_closest]

    return C, V, W_grid, optimal_c_init

In [4]:
C, V, W_grid, optimal_c_init = solve_consumption(
    W0=initial_wealth,
    r_ra=risk_adjusted_returns_per_year,
    r_tp=future_utility_discount_rate_per_year,
    gamma=GAMMA,
    T=time_horizon,
    n_grid=2_000,  # sets the maximum precision of wealth to search through
    c_grid_size=1001, # fractions of wealth to search through
)
print(f"Optimal c at t=1 for W0=${initial_wealth:,.2f}: {optimal_c_init:.4f}")

  0%|          | 0/20 [00:00<?, ?it/s]

Optimal c at t=1 for W0=$1,000,000.00: 0.0640


In [5]:
# Simulate forward
W_sim = [initial_wealth]
C_sim = []
cons_sim = []
for t in range(1, time_horizon + 1):
    i_closest = np.argmin(np.abs(W_grid - W_sim[-1]))
    c_star = C[t, i_closest]
    C_sim.append(c_star)    
    cons = c_star * W_sim[-1]
    cons_sim.append(cons)
    
    W_next = (W_sim[-1] - cons) * (1 + risk_adjusted_returns_per_year)
    W_sim.append(W_next)

In [6]:
df = pl.DataFrame(
    {"age": [65 + y for y in range(time_horizon+1)],
     "optimal_fractional_consumption": [None] + C_sim,
     "wealth": W_sim,
     "optimal_absolute_consumption": [None] + cons_sim
     }
)

with pl.Config(tbl_rows=50):
    display(df)

age,optimal_fractional_consumption,wealth,optimal_absolute_consumption
i64,f64,f64,f64
65,null,1e6,null
66,0.064,964080.0,64000.0
67,0.067,926471.2392,64593.36
68,0.065,892238.126912,60220.630548
69,0.07,854674.901769,62456.668884
70,0.072,816932.458106,61536.592927
71,0.074,779173.839893,60453.0019
72,0.083,735937.483517,64671.428711
73,0.091,689036.187693,66970.311
